# TP2 : Implementação do RSA - OAEP


## Função auxiliar

Foi utilizada uma função auxiliar para gerar um número primo dado o seu tamanho, retornando 
o primo com o respetivo tamanho indicado.

In [1]:
def rprime(l):
    """
    Gera um número primo
    Argumentos: 
        - l - tamanho do primo
    Valor de retorno: primo com o tamanho indicado
    """
    return random_prime(2**l-1,True,2**(l-1))

## Definição do problema
### Alinea a)

Para a parte inicial da primeira parte, tinhamos como objetivo implementar um esquema RSA - OAEP em que recebe como parâmetro de entrada um número de bits do módulo que irá ser utilizado para gerar as chaves pública e privada respetivamente.

### Alinea b)

OAEP - Optimal Asymmetric Encryption Padding - é um esquema de preenchimento frequentemente usado em conjunto com a criptografia RSA. Este algoritmo usa um par de oráculos aleatórios G e H para processar o texto sem formatação antes da criptografia assimétrica.

OAEP satisfaz os seguintes objetivos:

- Adiciona um elemento aleatório que possa ser usado para converter um esquema criptográfico determinístico (por exemplo, RSA tradicional) num esquema criptográfico probabilístico.
    
- Evita a descriptografia parcial de textos cifrados (ou outro vazamento de informações), garantindo que um adversário não pode recuperar nenhuma parte do texto simples.

### Alinea c)


Fujisaki-Okamoto apresentaram uma abordagem para transformar PKE's que fossem IND-CCA seguros noutros PKE's que fossem IND-CCA seguros. Este processo aumenta muito a complexidade espacial pois são criados criptogramas maiores para o mesmo parâmetro.

Esta transformação FOT segue o princípio de separar a geração e aleatoriedade do núcleo determinístico do algoritmo. O algoritmo determinístico é o porquê deste IND-CCA ser seguro.

É possível dizer que se a 1ª versão do PKE for IND-CPA seguro e se o OWN ("One Way Compressor") for um hash seguro, então a 2ª versão do PKE é um IND-CCA seguro.
Desta forma, assumindo que tanto o KEM como o OWN são seguros, podemos escolher entre ter um PKE mais eficiente mas com segurança mais fraca (IND-CPA) ou ter um PKE menos eficiente mas mais seguro (IND-CCA).

In [2]:
import random
import string
import hashlib

class RSA_OAEP:
    def __init__(self,nbits):
        self.nbits=int(nbits)
        
    def keygen(self):
        p=rprime(int(self.nbits/2) +1)
        q=rprime(int(self.nbits/2))
        while p<=2*q:
            p=rprime(int(self.nbits/2) +1)
            q=rprime(int(self.nbits/2))
        n=p*q #parâmetro n
        phin=(p-1)*(q-1) #Cálculo de phi de n para primos
        e=randint(2,phin)
        while gcd(phin,e)!=1:
            e=randint(2,phin) #e tem de ser tal que exista inverso módulo phi de n
        d=power_mod(e,-1,phin) #inverso de e
        PubKey=(n,e)
        PrivKey=d
        return PubKey,PrivKey
    
    def xor(self, data, mask):
        masked = b''
        ldata = len(data)
        lmask = len(mask)
        for i in range(max(ldata, lmask)):
            if i < ldata and i < lmask:
                masked += (data[i] ^^ mask[i]).to_bytes(1, byteorder='big')
            elif i < ldata:
                masked += data[i].to_bytes(1, byteorder='big')
            else:
                break
        return masked
    
    def encrypt(self, message, public_k, label):
        l= label.encode('utf-8')
        a =  hashlib.sha256(l).hexdigest()
        nm = str(a+message)            
        nm = nm.encode('utf-8')
        k = int(hashlib.sha256(nm).hexdigest(),16)
        (n,e) = public_k
        enc = power_mod(k , e, n)
        x = k.to_bytes(len(nm), byteorder='big')
        encf = self.xor(x,nm)
        return (enc,encf)
    
    def decrypt(self, ctxt, private_k, public_k, label):
        enc, encf = ctxt
        (n,e) = public_k
        k = power_mod(enc, private_k, n)
        m = int (k).to_bytes(len(encf), byteorder='big')
        txt = self.xor (m, encf)
        l= label.encode('utf-8')
        size = len (hashlib.sha256(l).hexdigest())
        a, message = txt[:size], txt[size:]
        msg = message.decode('ascii')
        if ctxt == self.encrypt(msg, public_k, label):
            return message.decode('ascii')
        else: return false



In [3]:
rsa_oaep = RSA_OAEP(512)
Pub,Priv=rsa_oaep.keygen()
msg = "Ola mundo" 
label= "EstCripto"
print ("Mensagem:", msg)

#para cifrar uma mensagem usa-se a chave pública do RSA
ctxt = rsa_oaep.encrypt(msg,Pub,label)
print ("CipherText:", ctxt)

txt = rsa_oaep.decrypt(ctxt,Priv, Pub, label)
print ("Message:", txt)

Mensagem: Ola mundo
CipherText: (1643718922565352104383403106865898091983261704864042424041212196720238784460826478557007264543821055867353666104125060936361276392011644737128563215093572, b'898f73a20b73f9963b6f7fde394930c3fd8657fbd#\xc121\x05m\x9f\xcd\xf0\xe3\x83\x9c_\xd5\xcc\x9e\x0etz.H1\xb428\xa0\xd2\xc5X\x86\xd7S')
Message: Ola mundo
